In [ ]:
import sys
!{sys.executable} -m pip install scikit-learn

In [ ]:
import os
import time
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from numpy.linalg import norm
from sklearn.utils.extmath import randomized_svd  # For randomized SVD
from memory_profiler import memory_usage

# --- File paths ---
base_path = r'G:\My Drive\NUS\NUS Y6S1\ME5311\PROJECT_2420_ME5311'
slp_path = os.path.join(base_path, 'slp.nc')
t2m_path = os.path.join(base_path, 't2m.nc')

# --- Load datasets ---
ds_slp = xr.open_dataset(slp_path)
ds_t2m = xr.open_dataset(t2m_path)

slp = ds_slp['msl'].values
t2m = ds_t2m['t2m'].values
timestamps = ds_slp['time'].values
lats = ds_slp['latitude'].values
longs = ds_slp['longitude'].values

# --- Reshape and center SLP ---
n_time, n_lat, n_lon = slp.shape
A_slp = slp.reshape(n_time, -1).T
A_mean_slp = A_slp.mean(axis=1, keepdims=True)
A_centered_slp = A_slp - A_mean_slp

In [ ]:
# --- Define number of singular values/vectors to compute ---
k = 500  # Number of singular values to compute, adjust based on your needs
n_oversamples = 10  # Extra samples for improved accuracy
n_iter = 5  # Number of power iterations for enhancing accuracy

# --- Define Randomized SVD function with monitoring ---
def perform_randomized_svd_with_monitoring(A, k, n_oversamples=10, n_iter=5):
    """Performs Randomized SVD with runtime and memory tracking"""
    print(f"Performing Randomized SVD on SLP data (k={k}, oversamples={n_oversamples}, iter={n_iter})...")
    
    def randomized_svd_task():
        global U_slp, S_slp, VT_slp
        
        # Use sklearn's randomized_svd
        U_slp, S_slp, VT_slp = randomized_svd(
            A, 
            n_components=k,
            n_oversamples=n_oversamples,
            n_iter=n_iter,
            random_state=42  # For reproducibility
        )

    # Monitor runtime and memory usage
    start = time.time()
    mem_usage = memory_usage(randomized_svd_task, max_usage=True)
    elapsed = time.time() - start
    
    return elapsed, mem_usage

# --- Run Randomized SVD with performance monitoring ---
elapsed_slp, peak_mem_slp = perform_randomized_svd_with_monitoring(
    A_centered_slp, k, n_oversamples, n_iter
)

# --- SVD result shapes (informational) ---
print(f"A shape: {A_slp.shape}")
print(f"U shape: {U_slp.shape}, S shape: {S_slp.shape}, VT shape: {VT_slp.shape}")
print(f"Randomized SVD with k={k}, oversamples={n_oversamples}, iterations={n_iter}")

In [ ]:
# --- Accuracy (Reconstruction Error) ---
def calculate_reconstruction_error(U, S, VT, A_original, A_mean):
    """Calculate reconstruction error using Frobenius norm"""
    # Create diagonal S matrix for matrix multiplication
    S_diag = np.diag(S)
    
    # Reconstruct the original matrix
    A_reconstructed = U @ S_diag @ VT + A_mean
    
    # Calculate relative error
    error = norm(A_original - A_reconstructed) / norm(A_original)
    return error, A_reconstructed

# --- Noise Robustness Test ---
def test_noise_robustness(A_centered, A_original, A_mean, k, n_oversamples=10, n_iter=5, noise_scale=0.01):
    """Test Randomized SVD robustness against Gaussian noise"""
    np.random.seed(0)  # For reproducibility
    noise = np.random.normal(scale=noise_scale, size=A_centered.shape)
    A_noisy = A_centered + noise
    
    # Perform Randomized SVD on noisy data
    U_noisy, S_noisy, VT_noisy = randomized_svd(
        A_noisy, 
        n_components=k,
        n_oversamples=n_oversamples,
        n_iter=n_iter,
        random_state=42
    )
    
    # Calculate reconstruction error with noise
    S_noisy_diag = np.diag(S_noisy)
    A_reconstructed_noisy = U_noisy @ S_noisy_diag @ VT_noisy + A_mean
    
    error = norm(A_original - A_reconstructed_noisy) / norm(A_original)
    return error

# Calculate reconstruction error
reconstruction_error, A_reconstructed = calculate_reconstruction_error(
    U_slp, S_slp, VT_slp, A_slp, A_mean_slp
)

# Test noise robustness
noise_error = test_noise_robustness(
    A_centered_slp, A_slp, A_mean_slp, k, n_oversamples, n_iter
)

# --- Report results ---
print("\n===== Randomized SVD Results for SLP =====")
print(f"Number of singular values/vectors: k = {k}")
print(f"Number of oversamples: {n_oversamples}")
print(f"Number of power iterations: {n_iter}")
print(f"Runtime: {elapsed_slp:.2f} seconds")
print(f"Peak memory usage: {peak_mem_slp:.2f} MiB")
print(f"Reconstruction error (Frobenius norm): {reconstruction_error:.6e}")
print(f"Noise robustness (error with Gaussian noise): {noise_error:.6e}")

# --- Cumulative energy (estimated) ---
total_energy = np.sum(S_slp**2)
cumulative_energy = np.cumsum(S_slp**2) / total_energy

# Note: Since randomized SVD only computes k values, we can only show the
# energy captured by these k components, not determine how many are needed for 90/95%
print(f"Energy captured by {k} components: {cumulative_energy[-1]:.6f} ({cumulative_energy[-1]*100:.2f}%)")

# --- Optional: Save results to file for later comparison ---
results = {
    "method": "Randomized SVD",
    "k_value": k,
    "n_oversamples": n_oversamples,
    "n_iterations": n_iter,
    "runtime": elapsed_slp,
    "memory_usage": peak_mem_slp,
    "reconstruction_error": float(reconstruction_error),
    "noise_robustness": float(noise_error),
    "energy_captured": float(cumulative_energy[-1]),
    "top_singular_values": S_slp[:10].tolist()  # Save first 10 singular values
}

# Save as JSON (optional)
import json
with open("randomized_svd_results.json", "w") as f:
    json.dump(results, f, indent=4)

# --- Plot singular value decay ---
plt.figure(figsize=(10, 6))
plt.semilogy(range(1, len(S_slp) + 1), S_slp, 'o-')
plt.title(f'Singular Value Decay (k={k}) - Randomized SVD')
plt.xlabel('Index')
plt.ylabel('Singular Value (log scale)')
plt.grid(True)
plt.savefig('randomized_svd_singular_values.png', dpi=300)
plt.show()

# --- Plot cumulative energy ---
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(S_slp) + 1), cumulative_energy, 'o-')
plt.title('Cumulative Energy vs. Number of Modes - Randomized SVD')
plt.xlabel('Number of Modes')
plt.ylabel('Cumulative Energy Fraction')
plt.grid(True)
plt.savefig('randomized_svd_cumulative_energy.png', dpi=300)
plt.show()

# --- Optional: Compare first few spatial modes with full SVD if available ---
# If you've run full SVD previously and saved U_full, you can compare:
# from numpy.linalg import svd
# _, _, VT_full = svd(A_centered_slp, full_matrices=False)
# 
# plt.figure(figsize=(12, 8))
# for i in range(min(4, k)):
#     plt.subplot(2, 2, i+1)
#     corr = np.abs(np.corrcoef(VT_slp[i], VT_full[i])[0, 1])
#     plt.plot(VT_full[i], label='Full SVD')
#     plt.plot(VT_slp[i], '--', label='Randomized SVD')
#     plt.title(f'Mode {i+1}, Correlation: {corr:.4f}')
#     plt.legend()
# plt.tight_layout()
# plt.savefig('randomized_vs_full_svd_modes.png', dpi=300)
# plt.show()